# Data Extraction

## Parsing Y- axis Ticks and Y- vals 
This code parses the x and y axes (horizontal and vertical axes) detection for the y-ticks, calculates the ratio to normalize the figure contents, specifically the bar plots with respect to the y-ticks and finally connects the evaluated y-values with their correspinding legends using clustering algorithm. 
* Note that all the figures recorded have their respective dois as a unique key of identification.

In [1]:
import cv2, imutils, re
import xlsxwriter, json, os
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from matplotlib import rcParams

In [2]:
# Directory of images to run the code on
img_dir = '../AllData/processed_images/Part2'

# Directory to save the output images
save_dir = 'out'

### Using json to parse the captions and dois for each image

In [3]:
with open('../data/urldata.json') as handler:
    urldata = json.load(handler)

In [4]:
with open('../data/doi_data.json') as doihandler:
    doidata = json.load(doihandler)

In [5]:
with open('../data/ocr-image-text.json') as ocrhandler:
    images_text = json.load(ocrhandler)

### Function to detect x-axis and y-axis for a plot

In [6]:
def findMaxConsecutiveOnes(nums) -> int:
    count = maxCount = 0
    
    for i in range(len(nums)):
        if nums[i] == 1:
            count += 1
        else:
            maxCount = max(count, maxCount)
            count = 0
                
    return max(count, maxCount)

In [7]:
def detectAxes(filepath, threshold=None, debug=False):
    if filepath is None:
        return None, None
    
    if threshold is None:
        threshold = 10
    
    image = cv2.imread(filepath)
    height, width, channels = image.shape
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    maxConsecutiveOnes = [findMaxConsecutiveOnes(gray[:, idx] < 200) for idx in range(width)]
    start_idx, maxindex, maxcount = 0, 0, max(maxConsecutiveOnes)
    while start_idx < width:
        if abs(maxConsecutiveOnes[start_idx] - maxcount) <= threshold:
            maxindex = start_idx
            break
            
        start_idx += 1
           
    yaxis = (maxindex, 0, maxindex, height)
    
    if debug:
        fig, ax = plt.subplots(1, 2)

        ax[0].imshow(image)

        ax[1].plot(maxConsecutiveOnes, color = 'k')
        ax[1].axhline(y = max(maxConsecutiveOnes) - 10, color = 'r', linestyle = 'dashed')
        ax[1].axhline(y = max(maxConsecutiveOnes) + 10, color = 'r', linestyle = 'dashed')
        ax[1].vlines(x = maxindex, ymin = 0.0, ymax = maxConsecutiveOnes[maxindex], color = 'b', linewidth = 4)

        plt.show()

    maxConsecutiveOnes = [findMaxConsecutiveOnes(gray[idx, :] < 200) for idx in range(height)]
    start_idx, maxindex, maxcount = 0, 0, max(maxConsecutiveOnes)
    while start_idx < height:
        if abs(maxConsecutiveOnes[start_idx] - maxcount) <= threshold:
            maxindex = start_idx
            
        start_idx += 1
            
    cv2.line(image, (0, maxindex), (width, maxindex),  (255, 0, 0), 2)
    xaxis = (0, maxindex, width, maxindex)
    
    if debug:
        rcParams['figure.figsize'] = 15, 8

        fig, ax = plt.subplots(1, 1)
        ax.imshow(image, aspect = 'auto')
        
    return xaxis, yaxis

In [8]:
def getProbableLabels(image, image_text, xaxis, yaxis):
    y_labels = []
    x_labels = []
    legends = []
    y_text_list = []
    
    height, width, channels = image.shape
    
    (x1, y1, x2, y2) = xaxis
    (x11, y11, x22, y22) = yaxis
    
    for text, (textx, texty, w, h) in image_text:
        text = text.strip()
            
        # To the left of y-axis and top of x-axis
        if (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == -1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == 1):
            
            numbers = re.findall(r'^[+-]?\d+(?:\.\d+)?$', text)
            if bool(numbers):
                y_labels.append((text, (textx, texty, w, h)))
            else:
                y_text_list.append((text, (textx, texty, w, h)))
            
        # To the right of y-axis and bottom of x-axis
        elif (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == 1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == -1):
            x_labels.append((text, (textx, texty, w, h)))
            
        # Top of x-axis and to the right of y-axis
        elif (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == -1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == -1):
            
            # Consider non-numeric only for legends
            legends.append((text, (textx, texty, w, h)))
    
    # Get the y-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(x11):
        count = 0
        current = []
        for index, (text, rect) in enumerate(y_labels):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(y_labels[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
    
    y_labels_list = maxList.copy()
    
    y_labels = []
    for text, (textx, texty, w, h) in maxList:
        y_labels.append(text)
        
    # Get the x-labels by finding the maximum
    # intersections with the sweeping line
    
    maxIntersection = 0
    maxList = []
    for i in range(y1, height):
        count = 0
        current = []
        for index, (text, rect) in enumerate(x_labels):
            if lineIntersectsRectY(i, rect):
                count += 1
                current.append(x_labels[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
    
    x_text = x_labels.copy()
    x_labels = []
    hmax = 0
    
    for text, (textx, texty, w, h) in maxList:
        x_labels.append(text)
        if texty + h > hmax:
            hmax = texty + h
    
    # Get possible x-text by moving from where we
    # left off in x-labels to the complete
    # height of the image.
    maxIntersection = 0
    maxList = []
    for i in range(hmax + 1, height):
        count = 0
        current = []
        for index, (text, rect) in enumerate(x_text):
            if lineIntersectsRectY(i, rect):
                count += 1
                current.append(x_text[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
    
    x_text = []
    for text, (textx, texty, w, h) in maxList:
        x_text.append(text)
    
    # Get possible legend text
    # For this, we need to search both top to
    # bottom and also from left to right.
    
    legends_and_numbers = mergeTextBoxes(legends)
    
    legends = []
    for text, (textx, texty, w, h) in legends_and_numbers:
        if not re.search(r'^[+-]?\d+(?:\.\d+)?[%]?$', text):
            legends.append((text, (textx, texty, w, h)))
    
    maxIntersection = 0
    maxList = []
    for i in range(y1):
        count = 0
        current = []
        for index, (text, rect) in enumerate(legends):
            if lineIntersectsRectY(i, rect):
                count += 1
                current.append(legends[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
            
    for i in range(x11, width):
        count = 0
        current = []
        for index, (text, rect) in enumerate(legends):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(legends[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
        
    legends = []
    for text, (textx, texty, w, h) in maxList:
        legends.append(text)
    
    return image, x_labels, x_text, y_labels, y_labels_list, y_text_list, legends, maxList

In [9]:
def lineIntersectsRectX(candx, rect):
    (x, y, w, h) = rect
    
    if x <= candx <= x + w:
        return True
    else:
        return False
    
def lineIntersectsRectY(candy, rect):
    (x, y, w, h) = rect
    
    if y <= candy <= y + h:
        return True
    else:
        return False

## Getting the Ratio for y-value matching 

Similar to the label detection logic, y-ticks are detected: 

Y-ticks 

*  Check only the numerical boxes which are to the left of y-axis and to the top of x-axis.
*  Run a line sweep from left end of the image to the y-axis position, and check when the sweeping line intersects with the maximum number of numerical boxes.
* The numerical boxes are then used as bounding boxes for calculating the y-ticks.

Ratio:
* Difference between the y-ticks is then calculated.  
* Only consider the mean difference between the y-ticks, rejecting the outliers from the calculated values.
* The final ratio is: <br> ``normalize_ratio := ticks_diff.mean() / y-ticks.mean()``.

In [10]:
def getRatio(path, xaxis, yaxis):
    list_text = []
    list_ticks = []
    
    filepath = img_dir + "/" + path.name
    
    image = cv2.imread(filepath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    height, width, channels = image.shape

    image_text = images_text[path.name]
    
    for text, (textx, texty, w, h) in image_text:
        text = text.strip()
                    
        (x1, y1, x2, y2) = xaxis
        (x11, y11, x22, y22) = yaxis
        
        # To the left of y-axis and top of x-axis
        if (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == -1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == 1):
            
            # Consider numeric only for ticks on y-axis
            numbers = re.findall(r'\d+(?:\.\d+)?', text)
            if bool(numbers):
                list_text.append((numbers[0], (textx, texty, w, h)))
                          
    # Get the y-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(x11):
        count = 0
        current = []
        for index, (text, rect) in enumerate(list_text):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(list_text[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
    
    # Get list of text and ticks
    list_text = []
    for text, (textx, texty, w, h) in maxList:
        list_text.append(float(text))
        list_ticks.append(float(texty + h))
        
    text_sorted = (sorted(list_text))
    ticks_sorted  = (sorted(list_ticks))
    
    ticks_diff = ([ticks_sorted[i] - ticks_sorted[i-1] for i in range(1, len(ticks_sorted))])
    text_diff = ([text_sorted[i] - text_sorted[i-1] for i in range(1, len(text_sorted))])
    print("[get text-to-tick ratio] ticks_diff: {0}, text_diff: {1}".format(ticks_diff, text_diff))
    
    # Detected text may not be perfect! Remove the outliers.
    ticks_diff = reject_outliers(np.array(ticks_diff), m=1)
    text_diff = reject_outliers(np.array(text_diff), m=1)
    print("[reject_outliers] ticks_diff: {0}, text_diff: {1}".format(ticks_diff, text_diff))
    
    normalize_ratio = np.array(text_diff).mean() / np.array(ticks_diff).mean()

    return text_sorted, normalize_ratio

In [11]:
def lineIntersectsRectX(candx, rect):
    (x, y, w, h) = rect
    
    if x <= candx <= x + w:
        return True
    else:
        return False
    
def lineIntersectsRectY(candy, rect):
    (x, y, w, h) = rect
    
    if y <= candy <= y + h:
        return True
    else:
        return False

In [12]:
def reject_outliers(data, m=1):
    return data[abs(data - np.mean(data)) <= m * np.std(data)]

## Writing to Excel workbook

In [13]:
def addToExcel(dataname, data, row):
    col = 0

    worksheet.write(row, col, dataname)
    for content in data:
        col += 1
        worksheet.write(row, col, content)

## Matching the ratio for final data extraction 

Y-val data:
* The height of each bounding box is calculated by cluster count estimation method. 
* This ratio previously calculated is then used to calculate the <br> ``y_values :=  v_value = normalize_ratio x height of bounding box``.

In [14]:
def mergeRects(contours):
    rects = []
    rectsUsed = []

    # Just initialize bounding rects and set all bools to false
    for cnt in contours:
        rects.append(cv2.boundingRect(cnt))
        rectsUsed.append(False)

    # Sort bounding rects by x coordinate
    def getXFromRect(item):
        return item[0]

    rects.sort(key = getXFromRect)

    # Array of accepted rects
    acceptedRects = []

    # Merge threshold for x coordinate distance
    xThr = 5
    yThr = 5

    # Iterate all initial bounding rects
    for supIdx, supVal in enumerate(rects):
        if (rectsUsed[supIdx] == False):

            # Initialize current rect
            currxMin = supVal[0]
            currxMax = supVal[0] + supVal[2]
            curryMin = supVal[1]
            curryMax = supVal[1] + supVal[3]

            # This bounding rect is used
            rectsUsed[supIdx] = True

            # Iterate all initial bounding rects
            # starting from the next
            for subIdx, subVal in enumerate(rects[(supIdx+1):], start = (supIdx+1)):

                # Initialize merge candidate
                candxMin = subVal[0]
                candxMax = subVal[0] + subVal[2]
                candyMin = subVal[1]
                candyMax = subVal[1] + subVal[3]

                # Check if x distance between current rect
                # and merge candidate is small enough
                if (candxMin <= currxMax + xThr):

                    if not nearbyRectangle((candxMin, candyMin, candxMax - candxMin, candyMax - candyMin),
                                           (currxMin, curryMin, currxMax - currxMin, curryMax - curryMin), yThr):
                        break

                    # Reset coordinates of current rect
                    currxMax = candxMax
                    curryMin = min(curryMin, candyMin)
                    curryMax = max(curryMax, candyMax)

                    # Merge candidate (bounding rect) is used
                    rectsUsed[subIdx] = True
                else:
                    break

            # No more merge candidates possible, accept current rect
            acceptedRects.append([currxMin, curryMin, currxMax - currxMin, curryMax - curryMin])

    #for rect in acceptedRects:
    #    img = cv2.rectangle(img, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (121, 11, 189), 2)
    
    return acceptedRects

In [15]:
def mergeTextBoxes(textboxes):
    rects = []
    rectsUsed = []
    
    # Just initialize bounding rects and set all bools to false
    for box in textboxes:
        rects.append(box)
        rectsUsed.append(False)

    # Sort bounding rects by x coordinate
    def getXFromRect(item):
        return item[1][0]
    
    def getYFromRect(item):
        return item[1][1]

    rects.sort(key = lambda x: (getYFromRect, getXFromRect))

    #print(rects)
    #print("----------------------------------")
    
    # Array of accepted rects
    acceptedRects = []

    # Merge threshold for x coordinate distance
    xThr = 10
    yThr = 0

    # Iterate all initial bounding rects
    for supIdx, supVal in enumerate(rects):
        if (rectsUsed[supIdx] == False):

            # Initialize current rect
            currxMin = supVal[1][0]
            currxMax = supVal[1][0] + supVal[1][2]
            curryMin = supVal[1][1]
            curryMax = supVal[1][1] + supVal[1][3]
            currText = supVal[0]

            # This bounding rect is used
            rectsUsed[supIdx] = True

            # Iterate all initial bounding rects
            # starting from the next
            for subIdx, subVal in enumerate(rects[(supIdx+1):], start = (supIdx+1)):

                # Initialize merge candidate
                candxMin = subVal[1][0]
                candxMax = subVal[1][0] + subVal[1][2]
                candyMin = subVal[1][1]
                candyMax = subVal[1][1] + subVal[1][3]
                candText = subVal[0]

                # Check if x distance between current rect
                # and merge candidate is small enough
                if (candxMin <= currxMax + xThr):

                    if not nearbyRectangle((candxMin, candyMin, candxMax - candxMin, candyMax - candyMin),
                                           (currxMin, curryMin, currxMax - currxMin, curryMax - curryMin), yThr):
                        break

                    # Reset coordinates of current rect
                    currxMax = candxMax
                    curryMin = min(curryMin, candyMin)
                    curryMax = max(curryMax, candyMax)
                    currText = currText + ' ' + candText
                    
                    # Merge candidate (bounding rect) is used
                    rectsUsed[subIdx] = True
                else:
                    break

            # No more merge candidates possible, accept current rect
            acceptedRects.append([currText, (currxMin, curryMin, currxMax - currxMin, curryMax - curryMin)])

    #for rect in acceptedRects:
    #    img = cv2.rectangle(img, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (121, 11, 189), 2)
    
    return acceptedRects

In [16]:
def nearbyRectangle(current, candidate, threshold):
    (currx, curry, currw, currh) = current
    (candx, candy, candw, candh) = candidate
    
    currxmin = currx
    currymin = curry
    currxmax = currx + currw
    currymax = curry + currh
    
    candxmin = candx
    candymin = candy
    candxmax = candx + candw
    candymax = candy + candh
    
    # If candidate is on top, and is close
    if candymax <= currymin and candymax + threshold >= currymin:
        return True
    
    # If candidate is on bottom and is close
    if candymin >= currymax and currymax + threshold >= candymin:
        return True
    
    # If intersecting at the top, merge it
    if candymax >= currymin and candymin <= currymin:
        return True
    
    # If intersecting at the bottom, merge it
    if currymax >= candymin and currymin <= candymin:
        return True
    
    # If intersecting on the sides or is inside, merge it
    if (candymin >= currymin and
        candymin <= currymax and
        candymax >= currymin and
        candymax <= currymax):
        return True
    
    return False

In [17]:
def euclidean(v1, v2):
    return sum((p - q) ** 2 for p, q in zip(v1, v2)) ** .5

def angle_between(p1, p2):
    
    deltaX = p1[0] - p2[0]
    deltaY = p1[1] - p2[1]

    return math.atan2(deltaY, deltaX) / math.pi * 180
    
def RectDist(rectA, rectB):
    (rectAx, rectAy, rectAw, rectAh) = rectA
    (rectBx, rectBy, rectBw, rectBh) = rectB
    
    angle = angle_between((rectAx + rectAw / 2, rectAy + rectAh / 2), (rectBx + rectBw / 2, rectBy + rectBh / 2))
    
    if (angle >= -5 and angle <= 5) or (angle <= -175 and angle >= -185) or (angle >= 175 and angle <= 185):
        if (rectBx > rectAx + rectAw):
            return euclidean([rectAx + rectAw / 2, rectAy + rectAh / 2], [rectBx + rectBw / 2, rectBy + rectBh / 2])
        else:
            return sys.maxsize
    else:
        return sys.maxsize

In [18]:
def getProbableYLabels(image, contours, xaxis, yaxis):
    y_labels = []
    
    height, width, channels = image.shape
    
    (x1, y1, x2, y2) = xaxis
    (x11, y11, x22, y22) = yaxis
    
    # Get the y-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(x11):
        count = 0
        current = []
        for index, rect in enumerate(contours):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(contours[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
                    
    return image, maxList

## Saving y-values in our data excel sheet 

In [19]:
import ast, cv2, re, sys
import math
import json

images = []
texts = []

def getYVal(img_dir):
    yValueDict = {}
    for index, path in enumerate(Path(img_dir).iterdir()):
        if path.name.endswith('.png') or path.name.endswith('.jpg'):
            filepath = img_dir + "/" + path.name
            img = cv2.imread(filepath)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            xaxis, yaxis = detectAxes(filepath)
            
            for (x1, y1, x2, y2) in [xaxis]:
                xaxis = (x1, y1, x2, y2)

            for (x1, y1, x2, y2) in [yaxis]:
                yaxis = (x1, y1, x2, y2)

            image_text = images_text[path.name]
            img, x_labels, _, _, _, _, legends, legendBoxes = getProbableLabels(img, image_text, xaxis, yaxis)
            try:
                list_text, normalize_ratio = getRatio(path, xaxis, yaxis)
                print("[getYVal] legends: {0}".format(legends))
                print("[{0}] path: {1}, ratio: {2}".format(index, path.name, normalize_ratio), end='\n\n')

                # Color detection
                h, w, _ = np.shape(img)

                image = img.reshape((h * w, 3))

                # Remove white and near-by pixels
                threshold = 5
                for r in range(255 - threshold, 256):
                    for g in range(255 - threshold, 256):
                        for b in range(255 - threshold, 256):
                            image = image[np.where((image != [r, g, b]).any(axis = 1))]

                values, counts = np.unique(image, axis = 0, return_counts = True)

                greater_counts = np.where(counts >= 100)
                values, counts = values[greater_counts], counts[greater_counts]

                sort_indices = np.argsort(-counts)
                values, counts = values[sort_indices], counts[sort_indices]

                groups = []
                groupcounts = []

                for idx, value in enumerate(values):
                    grouped = False

                    for groupid, group in enumerate(groups):
                        for member in group:
                            r, g, b = member
                            vr, vg, vb = value

                            if (abs(vr.astype(np.int16) - r.astype(np.int16)) <= 5 and
                                abs(vg.astype(np.int16) - g.astype(np.int16)) <= 5 and
                                abs(vb.astype(np.int16) - b.astype(np.int16)) <= 5):
                                group.append(value)
                                groupcounts[groupid] += counts[idx]
                                grouped = True
                                break

                        if grouped:
                            break

                    if not grouped:
                        groups.append([value])
                        groupcounts.append(counts[idx])

                groupcounts = np.array(groupcounts)
                sort_indices = np.argsort(-groupcounts)
                new_groups = [groups[i] for i in sort_indices]
                groups = new_groups
                
                for i in range(len(groups[:4])):

                    img = cv2.imread(img_dir + "/" + path.name)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                    mask = None
                    for value in groups[i]:
                        COLOR_MIN = np.array([value[0], value[1], value[2]], np.uint8)
                        COLOR_MAX = np.array([value[0], value[1], value[2]], np.uint8)

                        if mask is None:
                            mask = cv2.inRange(img, COLOR_MIN, COLOR_MAX)
                        else:
                            mask = mask | cv2.inRange(img, COLOR_MIN, COLOR_MAX)

                    image = cv2.bitwise_and(img, img, mask = mask)
                    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, (3, 3))

                    edged = cv2.Canny(image, 0, 250)
                    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    contours = [contour for contour in contours if cv2.contourArea(contour) > 0.]

                    # Remove noisy ones!
                    if len(contours) == 0 or len(contours) > 100:
                        continue

                    contours = [cv2.approxPolyDP(contour, 3, True) for contour in contours]

                    rects = mergeRects(contours)
                    textBoxes = []
                    legends = []
                    
                    for rectBox in rects:
                        min_distance = sys.maxsize
                        closestBox = None
                        legendtext = None

                        for text, textBox in legendBoxes:    
                            if RectDist(rectBox, textBox) < min_distance:
                                closestBox = textBox
                                min_distance = RectDist(rectBox, textBox)
                                legendtext = text

                        if closestBox is not None:
                            textBoxes.append(closestBox)

                        if legendtext is not None:
                            legends.append(legendtext)

                    list_len = []
                    
                    for rect in rects:
                        list_len.append(float(rect[3]))

                    # y-values will be a product of the normalize ratio and each length              
                    y_val = [round(l* normalize_ratio, 1) for l in list_len]

                    if path.name in yValueDict:
                        if legends:
                            yValueDict[path.name].append([legends[-1]])
                        else:
                            yValueDict[path.name].append([''])
                            
                        yValueDict[path.name].append(y_val)
                    else:
                        if legends:
                            yValueDict[path.name] = [legends[-1]]
                        else:
                            yValueDict[path.name] = ['']
                            
                        yValueDict[path.name].append(y_val)
                        
                    for box in textBoxes:
                        (x, y, w, h) = box
                        
            except Exception as e:
                print(e)
                continue
            
    return yValueDict

In [20]:
yValueDict = getYVal(img_dir)

[get text-to-tick ratio] ticks_diff: [44.0, 45.0, 47.0, 44.0, 46.0, 44.0], text_diff: [15.0, 15.0, 15.0, 15.0, 15.0, 15.0]
[reject_outliers] ticks_diff: [44. 45. 44. 46. 44.], text_diff: [15. 15. 15. 15. 15. 15.]
[getYVal] legends: ['Aqueous Phase', 'Gaseous Phase', 'Solid Residue']
[0] path: 08813216-Figure1-1.png, ratio: 0.3363228699551569

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[getYVal] legends: ['A', 'B']
[1] path: 08813216-Figure4-1.png, ratio: nan



C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\users\dsand\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:62: RuntimeWar

[get text-to-tick ratio] ticks_diff: [34.0, 34.0, 33.0, 34.0, 35.0, 34.0, 33.0, 35.0, 33.0], text_diff: [0.1, 0.09999999999999998, 0.10000000000000003, 0.09999999999999998, 0.09999999999999998, 0.09999999999999998, 0.10000000000000009, 0.09999999999999998, 0.09999999999999998]
[reject_outliers] ticks_diff: [34. 34. 34. 34.], text_diff: [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[getYVal] legends: ['SR', 'HSP', 'MSP', 'WSP']
[2] path: 1-s2.0-S0009250913008336-main-Figure10-1.png, ratio: 0.0029411764705882353

[get text-to-tick ratio] ticks_diff: [33.0, 34.0, 31.0, 34.0, 34.0, 34.0, 31.0, 35.0, 32.0], text_diff: [0.1, 0.09999999999999998, 0.10000000000000003, 0.09999999999999998, 0.09999999999999998, 0.09999999999999998, 0.10000000000000009, 0.09999999999999998, 0.09999999999999998]
[reject_outliers] ticks_diff: [33. 34. 34. 34. 34. 32.], text_diff: [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[getYVal] legends: ['0.9-1.0cm', 'SR', 'HSP', 'MSP', 'WSP']
[3] path: 1-s2.0-S0009250913008336-main-Figure11-1.png,

[get text-to-tick ratio] ticks_diff: [65.0, 189.0], text_diff: [60.0, 20.0]
[reject_outliers] ticks_diff: [ 65. 189.], text_diff: [60. 20.]
[getYVal] legends: ['Conversion', 'Yield of paraffin']
[25] path: 1-s2.0-S0016236115011345-main-Figure2-1.png, ratio: 0.31496062992125984

[get text-to-tick ratio] ticks_diff: [70.0, 70.0], text_diff: [10.0, 10.0]
[reject_outliers] ticks_diff: [70. 70.], text_diff: [10. 10.]
[getYVal] legends: ['Raw Algae', 'Crude Lipids', 'Residual Solids', 'AC-Purified Lipids', 'K10-Purified Lipids']
[26] path: 1-s2.0-S0016236116302447-main-Figure1-1.png, ratio: 0.14285714285714285

[get text-to-tick ratio] ticks_diff: [43.0, 43.0, 44.0, 44.0, 42.0, 44.0, 42.0, 45.0], text_diff: [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [43. 43. 44. 44. 44.], text_diff: [10. 10. 10. 10. 10. 10. 10. 10.]
[getYVal] legends: ['Raw Algae', 'Residual Solids', 'Crude Lipids', 'AC-Purified Lipids']
[27] path: 1-s2.0-S0016236116302447-main-Figure2-1.p

[get text-to-tick ratio] ticks_diff: [51.0, 50.0, 51.0, 51.0, 52.0, 49.0, 51.0, 51.0, 50.0, 50.0], text_diff: [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]
[reject_outliers] ticks_diff: [51. 50. 51. 51. 51. 51. 50. 50.], text_diff: [5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
[getYVal] legends: []
[49] path: 1-s2.0-S0016236118304496-main-Figure5-1.png, ratio: 0.09876543209876543

[get text-to-tick ratio] ticks_diff: [94.0, 93.0, 93.0, 94.0, 94.0], text_diff: [20.0, 20.0, 20.0, 20.0, 20.0]
[reject_outliers] ticks_diff: [94. 94. 94.], text_diff: [20. 20. 20. 20. 20.]
[getYVal] legends: ['Chlorella', 'Pavlova Spirulina platensis', 'cruentum Isochrysis Entermorpha Nannochloropsis', '& Nannochloropsis Nannochloropsis Nannochloropsis Nannochloropsist Nannochloropsis Nannochloropsis Spirulina', 'Porphyridium Spirulinad']
[50] path: 1-s2.0-S0016236118304502-main-Figure4-1.png, ratio: 0.2127659574468085

[get text-to-tick ratio] ticks_diff: [66.0, 63.0, 132.0, 66.0], text_diff: [0.0, 0.2, 0.39999999999

...

[get text-to-tick ratio] ticks_diff: [26.0, 27.0, 29.0, 27.0, 28.0, 28.0, 28.0, 27.0], text_diff: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[reject_outliers] ticks_diff: [27. 27. 28. 28. 28. 27.], text_diff: [1. 1. 1. 1. 1. 1. 1. 1.]
[getYVal] legends: []
[1122] path: ef5018708-Figure5-1.png, ratio: 0.03636363636363636

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[getYVal] legends: []
[1123] path: ef502574b-Figure6-1.png, ratio: nan

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[getYVal] legends: []
[1124] path: ef502773w-Figure2-1.png, ratio: nan

[get text-to-tick ratio] ticks_diff: [39.0, 38.0, 39.0, 39.0], text_diff: [5.0, 5.0, 5.0, 5.0]
[reject_outliers] ticks_diff: [39. 39. 39.], text_diff: [5. 5. 5. 5.]
[getYVal] legends: ['CH4', 'H2']
[1125] path: ef700497d-Figure6-1.png, ratio: 0.1282051282051282

[get text-to-tick ratio] ticks_diff: [80.0, 160.0, 82.0], text_dif

[get text-to-tick ratio] ticks_diff: [27.0, 31.0, 28.0, 31.0, 177.0, 27.0], text_diff: [10.0, 60.0, 10.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [27. 31. 28. 31. 27.], text_diff: [10. 10. 10. 10. 10.]
[getYVal] legends: []
[1149] path: ep.12490-Figure2-1.png, ratio: 0.3472222222222222

[get text-to-tick ratio] ticks_diff: [31.0, 29.0, 207.0], text_diff: [14.0, 2.0, 2.0]
[reject_outliers] ticks_diff: [31. 29.], text_diff: [2. 2.]
[getYVal] legends: ['SPME', 'DAE']
[1150] path: ep.12490-Figure4-1.png, ratio: 0.06666666666666667

[get text-to-tick ratio] ticks_diff: [27.0, 28.0, 28.0, 28.0, 28.0, 27.0, 28.0, 29.0, 26.0], text_diff: [0.1, 0.1, 0.09999999999999998, 0.10000000000000003, 0.09999999999999998, 0.09999999999999998, 0.09999999999999998, 0.10000000000000009, 0.09999999999999998]
[reject_outliers] ticks_diff: [27. 28. 28. 28. 28. 27. 28.], text_diff: [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[getYVal] legends: ['Liquid', 'Gas']
[1151] path: ep.12713-Figure10-1.png, ratio: 0.003608

[get text-to-tick ratio] ticks_diff: [38.0, 40.0, 37.0, 39.0, 39.0, 38.0, 39.0], text_diff: [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [38. 39. 39. 38. 39.], text_diff: [10. 10. 10. 10. 10. 10. 10.]
[getYVal] legends: ['No H2', 'High-Pressurel H2']
[1177] path: ie100758s-Figure2-1.png, ratio: 0.25906735751295334

[get text-to-tick ratio] ticks_diff: [57.0, 54.0, 54.0, 55.0], text_diff: [20.0, 20.0, 20.0, 20.0]
[reject_outliers] ticks_diff: [54. 54. 55.], text_diff: [20. 20. 20. 20.]
[getYVal] legends: ['N2', 'C2H6', 'C2H4', 'CH4', '0CO2']
[1178] path: ie100758s-Figure8-1.png, ratio: 0.36809815950920244

[get text-to-tick ratio] ticks_diff: [53.0, 162.0, 54.0], text_diff: [20.0, 60.0, 20.0]
[reject_outliers] ticks_diff: [53. 54.], text_diff: [20. 20.]
[getYVal] legends: ['C2H6', 'BCH4', 'C02']
[1179] path: ie100758s-Figure9-1.png, ratio: 0.37383177570093457

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: 

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[getYVal] legends: []
[1206] path: Onoja2019_Article_OilPalmElaeisGuineensisBiomass-Figure3-1.png, ratio: nan

[get text-to-tick ratio] ticks_diff: [50.0, 51.0, 51.0, 53.0], text_diff: [5.0, 5.0, 5.0, 5.0]
[reject_outliers] ticks_diff: [51. 51.], text_diff: [5. 5. 5. 5.]
[getYVal] legends: []
[1207] path: Onwudili2014_Chapter_HydrothermalGasificationOfBiom-Figure10.10-1.png, ratio: 0.09803921568627451

[get text-to-tick ratio] ticks_diff: [51.0, 51.0, 51.0, 53.0, 51.0, 52.0], text_diff: [5.0, 5.0, 5.0, 5.0, 5.0, 5.0]
[reject_outliers] ticks_diff: [51. 51. 51. 51. 52.], text_diff: [5. 5. 5. 5. 5. 5.]
[getYVal] legends: []
[1208] path: Onwudili2014_Chapter_HydrothermalGasificationOfBiom-Figure10.11-1.png, ratio: 0.09765625

[get text-to-tick ratio] ticks_diff: [63.0, 62.0, 62.0, 63.0, 61.0], text_diff: [10.0, 10.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [62. 62.], text_diff: [1

[get text-to-tick ratio] ticks_diff: [59.0, 57.0, 60.0, 60.0, 59.0], text_diff: [10.0, 10.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [59. 60. 60. 59.], text_diff: [10. 10. 10. 10. 10.]
[getYVal] legends: ['THERMAL', 'KOH', 'K ,CO, 2']
[1233] path: Singh2015_Article_HydrothermalLiquefactionOfAgri-Figure5-1.png, ratio: 0.16806722689075632

[get text-to-tick ratio] ticks_diff: [258.0], text_diff: [40.0]
[reject_outliers] ticks_diff: [258.], text_diff: [40.]
[getYVal] legends: ['THERMAL', 'KOH', 'K K,CO,']
[1234] path: Singh2015_Article_HydrothermalLiquefactionOfAgri-Figure6-1.png, ratio: 0.15503875968992248

[get text-to-tick ratio] ticks_diff: [70.0, 72.0, 72.0], text_diff: [10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [72. 72.], text_diff: [10. 10. 10.]
[getYVal] legends: ['THERMAL', 'KOH', 'K,CO,']
[1235] path: Singh2015_Article_HydrothermalLiquefactionOfAgri-Figure7-1.png, ratio: 0.1388888888888889

[get text-to-tick ratio] ticks_diff: [256.0], text_diff: [40.0]
[reject_out

In [21]:
workbook = xlsxwriter.Workbook('../results/FigureData1.xlsx')

for index, path in enumerate(Path(img_dir).iterdir()):
    if path.name.endswith('.png') or path.name.endswith('.jpg') or path.name.endswith('.jpeg'):
        if path.name in yValueDict:
            filepath = img_dir + "/" + path.name
            print("[{0}] path: {1}".format(index, path.name))
            
            image = cv2.imread(filepath)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            height, width, channels = image.shape
            xaxis, yaxis = detectAxes(filepath)
            y_text = []

            for (x1, y1, x2, y2) in [xaxis]:
                xaxis = (x1, y1, x2, y2)

            for (x1, y1, x2, y2) in [yaxis]:
                yaxis = (x1, y1, x2, y2)
                
            image_text = images_text[path.name]
            image, x_labels, x_text, y_labels, y_labels_list, y_text_list, legends, _ = getProbableLabels(image,
                                                                                                  image_text,
                                                                                                  xaxis,
                                                                                                  yaxis)
            
            # Sort bounding rects by y coordinate
            def getYFromRect(item):
                return item[1][1]

            y_labels_list.sort(key = getYFromRect)
            y_text_list.sort(key = getYFromRect, reverse=True)
            
            for text, (textx, texty, w, h) in y_text_list:
                y_text.append(text)

            # Append doi values for each image.
            if path.name[:-16]+'.pdf' in doidata['Dois']:
                doi = doidata['Dois'][path.name[:-16]+'.pdf']
            else:
                doi = ''
             
            # Write to Excel
            worksheet = workbook.add_worksheet()            
            
            addToExcel("doi", doi, 0)
            addToExcel("file name", [path.name], 1)
            addToExcel("x-text", x_text, 2)
            addToExcel("x-labels", x_labels, 3)
            addToExcel("y-text", y_text, 4)
            addToExcel("y-labels", y_labels, 5)
            addToExcel("legends", legends, 6)
            
            for col_num, data in enumerate(yValueDict[path.name]):
                print(col_num, data)
                if type(data) != list:
                    worksheet.write(9 + col_num, 0, "Y-values:" + "\n\n" + str(data))
                else:
                    for pos, d in enumerate(data):
                        worksheet.write(9 + col_num, pos, str(d))

            # Print the output here!
            print("file name    :  ", path.name)
            print("doi          :  ", doi)
            print("x-text       :  ", x_text)
            print("x-labels     :  ", x_labels)
            print("y-text       :  ", y_text)
            print("y-labels     :  ", y_labels)
            print("legends      :  ", legends)
            print("Y- Values    :  ", ' '.join([str(elem) for elem in yValueDict[path.name]]), end = "\n\n")

            # Insert the image
            worksheet.insert_image('J21', filepath)

# Close the excel workbook!
workbook.close()

[0] path: 08813216-Figure1-1.png
0 Aqueous Phase
1 [4.4, 35.3, 0.7, 35.0, 28.9, 30.9, 37.7]
2 ['Solid Residue']
3 [4.7, 26.9, 25.2, 26.6, 22.2, 30.3, 27.2]
4 ['Gaseous Phase']
5 [17.2, 20.9, 25.9, 4.7, 1.7, 20.2, 19.5]
6 ['Solid Residue']
7 [21.2, 18.8, 17.8, 18.5, 15.5, 4.7, 15.5]
file name    :   08813216-Figure1-1.png
doi          :   
x-text       :   []
x-labels     :   ['270 0', "290'C", '310 C', "330'C", '350']
y-text       :   ['perce', 'of']
y-labels     :   ['105', '90', '75', '60', '45', '30', '15']
legends      :   ['Aqueous Phase', 'Gaseous Phase', 'Solid Residue']
Y- Values    :   Aqueous Phase [4.4, 35.3, 0.7, 35.0, 28.9, 30.9, 37.7] ['Solid Residue'] [4.7, 26.9, 25.2, 26.6, 22.2, 30.3, 27.2] ['Gaseous Phase'] [17.2, 20.9, 25.9, 4.7, 1.7, 20.2, 19.5] ['Solid Residue'] [21.2, 18.8, 17.8, 18.5, 15.5, 4.7, 15.5]

[1] path: 08813216-Figure4-1.png
0 
1 [nan]
2 ['']
3 [nan, nan]
4 ['B']
5 [nan]
file name    :   08813216-Figure4-1.png
doi          :   
x-text       :   ['Peak',

0 C-20:0. 20:1, 20:2
1 [0.0, 0.1, 0.1, 0.1, 0.2, 0.1, 15.1, 0.1, 4.0, 0.1, 0.4, 1.3, 0.5, 0.1, 0.2, 0.1, 0.2, 0.1, 0.4, 0.1, 0.1, 0.1, 0.1, 0.4, 0.4, 4.4, 0.1, 0.5, 1.5, 0.5, 0.1, 0.2, 0.1, 0.3, 0.1, 0.1, 0.1, 0.1, 0.1, 4.1, 0.1, 1.6, 0.1, 0.1, 0.1, 3.9, 0.1, 0.1, 1.7, 0.1]
2 ['C-20:0. 20:1, 20:2']
3 [0.1, 0.3, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.1, 0.6, 0.1, 0.1, 0.1, 0.2, 0.3, 0.2, 0.5, 0.5, 0.4, 0.5, 0.2, 0.5, 0.5, 0.1, 0.5, 0.0, 0.2, 0.2, 0.1, 0.2, 0.4, 0.3, 0.2, 0.4, 0.2, 0.8]
file name    :   1-s2.0-S0016236114000477-main-Figure8-1.png
doi          :   
x-text       :   ['Gold', 'Gold', 'ZrO2']
x-labels     :   ['K2CO3', 'K2CO3', 'K2C03', 'K2C03']
y-text       :   ['Fatty', 'acid', 'content']
y-labels     :   ['14', '12', '10', '8', '4', '2']
legends      :   ['C-16:0', 'C-18:0. 18:1, 18:2', 'C-20:0. 20:1, 20:2']
Y- Values    :   C-20:0. 20:1, 20:2 [0.0, 0.1, 0.1, 0.1, 0.2, 0.1, 15.1, 0.1, 4.0, 0.1, 0.4, 1.3, 0.5, 0.1, 0.2, 0.1, 0.2, 0.1, 0.4, 

0 K10-Purified Lipids
1 [0.4, 1.9, 23.3, 1.3, 3.3, 1.0, 8.9, 21.0, 33.6]
file name    :   1-s2.0-S0016236116302447-main-Figure1-1.png
doi          :   
x-text       :   ['Fatty', 'Acid', 'Chains']
x-labels     :   ['C10:0', 'C14:0', 'C15:0', 'C16:0', 'C16:1', 'C17:1', 'C18:0', 'C18:1c', 'C18:2c', 'C18:3n6']
y-text       :   ['com', '(wt%)']
y-labels     :   ['50', '40', '30']
legends      :   ['Raw Algae', 'Crude Lipids', 'Residual Solids', 'AC-Purified Lipids', 'K10-Purified Lipids']
Y- Values    :   K10-Purified Lipids [0.4, 1.9, 23.3, 1.3, 3.3, 1.0, 8.9, 21.0, 33.6]

[27] path: 1-s2.0-S0016236116302447-main-Figure2-1.png
0 Residual Solids
1 [4.6, 30.3, 32.8, 8.5, 5.3, 2.3, 47.7]
2 ['AC-Purified Lipids']
3 [2.1, 3.4, 82.3, 11.7, 2.3, 2.5]
file name    :   1-s2.0-S0016236116302447-main-Figure2-1.png
doi          :   
x-text       :   []
x-labels     :   ['Sulfur', 'Carbon', 'Moisture', 'Nitrogen', 'Oxygen', 'Hydrogen']
y-text       :   ['ConCEn']
y-labels     :   ['90', '80', '70', '6

0 Dilute acid
1 [1.8, 41.4, 2.7, 5.2, 26.4]
2 ['']
3 [2.0, 0.5, 0.5, 0.5, 0.5, 59.8, 2.0, 1.8, 0.9, 1.1, 1.4, 1.8, 1.4, 1.4]
4 ['Dilute acid']
5 [1.4, 0.9, 1.8]
file name    :   1-s2.0-S0016236117307172-main-Figure2-1.png
doi          :   
x-text       :   []
x-labels     :   ['Mass', 'loss', 'Ash', 'Carbohydrates', 'Protein']
y-text       :   ['Yie']
y-labels     :   ['60', '50', '40', '30', '20', '10']
legends      :   ['Dilute acid', '-treatment']
Y- Values    :   Dilute acid [1.8, 41.4, 2.7, 5.2, 26.4] [''] [2.0, 0.5, 0.5, 0.5, 0.5, 59.8, 2.0, 1.8, 0.9, 1.1, 1.4, 1.8, 1.4, 1.4] ['Dilute acid'] [1.4, 0.9, 1.8]

[37] path: 1-s2.0-S0016236117307172-main-Figure3-1.png
0 
1 [1.3, 1.3, 0.8, 2.5, 0.8, 1.7, 2.1, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 122.6, 4.2, 3.0, 2.1, 2.1, 4.7, 4.2, 4.2, 2.1]
file name    :   1-s2.0-S0016236117307172-main-Figure3-1.png
doi          :   
x-text       :   []
x-labels     :   ['Raw', 'microalgae', 'PA', '(NaOH/urea)', 'PA', '(dilute', 'acid)']


0 
1 [56.8, 8.2, 0.2, 63.9, 1.1, 0.2, 64.8, 67.0, 0.9, 65.0, 0.2]
2 ['']
3 [32.3, 22.8, 0.2, 21.6, 0.4, 14.6, 20.3, 0.7]
file name    :   1-s2.0-S0016236118304496-main-Figure4-1.png
doi          :   
x-text       :   ['Bio-crude', 'Solid', 'residue', 'Gas']
x-labels     :   ['No', 'catalyst', 'HZSM-5', 'Co/HZSM-5', 'Co-Zn/HZSM-5', 'Zn/HZSM-5']
y-text       :   ['yields', '(W']
y-labels     :   ['70', '60', '50', '40', '30', '20', '10']
legends      :   []
Y- Values    :    [56.8, 8.2, 0.2, 63.9, 1.1, 0.2, 64.8, 67.0, 0.9, 65.0, 0.2] [''] [32.3, 22.8, 0.2, 21.6, 0.4, 14.6, 20.3, 0.7]

[49] path: 1-s2.0-S0016236118304496-main-Figure5-1.png
0 
1 [35.3, 39.6, 0.5, 42.4, 46.7, 43.3]
2 ['']
3 [33.8, 25.1, 0.8, 0.1, 18.3, 10.6, 17.2]
file name    :   1-s2.0-S0016236118304496-main-Figure5-1.png
doi          :   
x-text       :   ['H2C02C0', 'CH4', 'C2-C3']
x-labels     :   ['catalyst', 'HZSM-5', 'Co/HZSM-5', 'Co-Zn/HZSM-5', 'Zn/HZSM-5']
y-text       :   ['(vo']
y-labels     :   ['50', '45', '4

...

0 
1 [nan, nan, nan, nan, nan, nan, nan, nan]
file name    :   Kumar-Pant2016_Article_BiorefinerySolidCakeWasteToBio-Figure2-1.png
doi          :   
x-text       :   []
x-labels     :   ['Standard', 'size', 'effect']
y-text       :   ['Temperature', 'Time', 'Pressure', 'to', 'ratio', 'Water', 'biomass', 'pressure', 'Initial', 'Heating', 'rate', 'Particle', 'size']
y-labels     :   []
legends      :   ['p=0.05']
Y- Values    :    [nan, nan, nan, nan, nan, nan, nan, nan]

[1195] path: Life cycle analysis of low temperature hydrothermal treatment pathway to produce biodiesel from microalgae-Figure4-1.png
0 
1 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
2 ['']
3 [nan, nan, nan, nan]
4 ['']
5 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
6 ['']
7 [

0 
1 [8.1, 12.3, 15.0, 21.3]
file name    :   Onwudili2014_Chapter_HydrothermalGasificationOfBiom-Figure10.10-1.png
doi          :   
x-text       :   ['Carbon', 'Activated', 'Carbon', 'Charcoal', 'Charcoal']
x-labels     :   ['Coal', 'Activated', 'Coconut', 'Shell', 'Macadamia', 'Shell', 'Spruce', 'wood']
y-text       :   ['mol/kg']
y-labels     :   ['25', '20', '15', '10', '5']
legends      :   []
Y- Values    :    [8.1, 12.3, 15.0, 21.3]

[1208] path: Onwudili2014_Chapter_HydrothermalGasificationOfBiom-Figure10.11-1.png
0 
1 [0.1, 0.5, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.5, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.4, 2.4, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 12.5, 0.3, 0.5, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 1.0, 0.1, 0.5, 10.9, 0.1, 0.3, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 31.2, 0.4, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 15.5, 0.1, 0.1, 0.5, 0.1, 0.1, 0.1, 0.1, 0.1]
file name    :  

0 
1 [3.2, 2.2, 17.2, 4.5, 6.0, 9.8, 5.2, 10.2, 5.5, 5.8, 7.8, 8.5, 2.0, 6.8, 9.0, 7.2, 7.8]
file name    :   s40643-018-0233-5-Figure3-1.png
doi          :   
x-text       :   []
x-labels     :   ['et al', 'et al', 'et al', 'et al', '-BsW)', 'et al', 'et al', 'etal', 'etal', 'et al', 'IPBR-', 'R(H.p', '.&ma', 'IOp-fw', 'Khoo', 'lOp', 'BR-EW', 'tal', 'Collet', 'anfer', 'Frank', 'Lardor', 'yorquer', 'bEnnio']
y-text       :   ['inpit', 'dry', 'algae]']
y-labels     :   ['50', '45', '40', '35', '30', '25', '20', '15', '10', '5']
legends      :   []
Y- Values    :    [3.2, 2.2, 17.2, 4.5, 6.0, 9.8, 5.2, 10.2, 5.5, 5.8, 7.8, 8.5, 2.0, 6.8, 9.0, 7.2, 7.8]

[1224] path: Saber2018_Article_UltrasonicPretreatmentForLow-t-Figure3-1.png
0 
1 [17.0, 23.4, 26.4]
2 ['']
3 [15.0, 20.8, 24.3]
4 ['']
5 [11.4, 16.6, 19.6, 0.4]
file name    :   Saber2018_Article_UltrasonicPretreatmentForLow-t-Figure3-1.png
doi          :   
x-text       :   ['Temperature (c)']
x-labels     :   ['210', '230', '250']
y-tex

0 N. gaditana
1 [1.7, 16.6, 20.9, 25.7, 25.2, 27.2, 47.1]
2 ['']
3 [0.3, 0.3, 0.9, 1.0, 2.1, 9.1, 0.9, 2.1, 1.2, 0.3, 13.1, 2.1, 0.5, 0.3, 21.0, 0.9, 0.5, 0.3, 1.2, 10.0, 2.9, 2.4, 0.9, 32.8, 32.9, 0.5, 2.1]
file name    :   Sudasinghe2015_Article_Temperature-DependentLipidConv-Figure1-1.png
doi          :   
x-text       :   ['HTL', 'Reaction', 'Temperature', '(oC)']
x-labels     :   ['180', '200', '225', '250', '275', '300']
y-text       :   ['o', '(%,', 'W']
y-labels     :   ['60', '50', '40', '30', '20', '10']
legends      :   ['8 Chlorella sp.', 'N. gaditana']
Y- Values    :   N. gaditana [1.7, 16.6, 20.9, 25.7, 25.2, 27.2, 47.1] [''] [0.3, 0.3, 0.9, 1.0, 2.1, 9.1, 0.9, 2.1, 1.2, 0.3, 13.1, 2.1, 0.5, 0.3, 21.0, 0.9, 0.5, 0.3, 1.2, 10.0, 2.9, 2.4, 0.9, 32.8, 32.9, 0.5, 2.1]

[1240] path: Sudasinghe2015_Article_Temperature-DependentLipidConv-Figure3-1.png
0 
1 [14.0, 4.2, 1.8, 30.0, 10.6, 4.8, 0.8, 10.3, 7.6, 3.2, 0.2, 1.0]
2 ['']
3 [10.3, 3.9, 1.8, 26.4, 12.1, 7.0, 2.4, 6.9, 9.3, 5

0 costs
1 [0.3, 3.6, 0.3, 0.3, 0.3, 0.6, 0.3, 0.3, 0.3, 0.3, 0.3, 0.8, 1.4, 2.5, 0.3, 18.5, 1.1, 2.2, 0.3, 0.3, 1.4, 0.6, 2.5, 1.9, 0.3, 0.6, 0.3, 1.1, 0.3, 0.3, 19.9, 0.3, 0.3, 3.3, 0.3, 0.3, 0.6, 1.4, 0.3, 0.3, 1.4, 2.8, 0.3, 0.6, 2.5, 0.3, 0.3, 0.3, 0.3, 0.3, 21.0, 0.3, 1.4, 1.4, 0.3, 1.4, 1.9, 0.3, 0.3, 0.3, 1.9, 0.3, 0.3, 2.5]
2 ['']
3 [17.1, 17.9, 18.5, 3.3]
4 ['']
5 [3.3, 7.4, 8.3, 8.8]
6 ['']
7 [5.2, 6.1, 3.3, 6.3]
file name    :   wene.319-Figure6-1.png
doi          :   
x-text       :   []
x-labels     :   ['LLE', 'HDO', 'reators', 'Chemical', 'storage', 'Credits']
y-text       :   ['SMm', 'operating', 'Net']
y-labels     :   ['50', '40', '30', '20', '10', '-20']
legends      :   ['costs']
Y- Values    :   costs [0.3, 3.6, 0.3, 0.3, 0.3, 0.6, 0.3, 0.3, 0.3, 0.3, 0.3, 0.8, 1.4, 2.5, 0.3, 18.5, 1.1, 2.2, 0.3, 0.3, 1.4, 0.6, 2.5, 1.9, 0.3, 0.6, 0.3, 1.1, 0.3, 0.3, 19.9, 0.3, 0.3, 3.3, 0.3, 0.3, 0.6, 1.4, 0.3, 0.3, 1.4, 2.8, 0.3, 0.6, 2.5, 0.3, 0.3, 0.3, 0.3, 0.3, 21.0, 0.3, 1.4,